In [66]:
# !pip install PyMuPDF

In [67]:
import pymupdf
import datetime

from langchain_core.tools import Tool

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage
import json
from langchain_core.agents import AgentAction, AgentFinish
from langchain.agents.output_parsers.tools import (
    ToolAgentAction,
    parse_ai_message_to_tool_action,
)

In [68]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    # localhost so does not matter
    secret_key="sk-lf-c650de69-a630-4c5a-b5bc-93a9745f663d",
    public_key="pk-lf-344d1105-8258-461a-bbc8-5cb5330389a9",
    host="http://localhost:3000",
)

In [69]:
doc = None

In [70]:
llm = ChatOpenAI(
    model="gpt-4o",
    max_retries=2,
    temperature=0.25,
)

In [71]:
def open_pdf(pdf_name):
    global doc
    doc = pymupdf.open(pdf_name)
    return f"Opened {pdf_name} pdf file for analysis. The document has {len(doc)} pages."

open_pdf_tool = Tool.from_function(
    name='open_pdf_tool',
    func=open_pdf,
    description="Opens a PDF file for analysis. Takes in the pdf file name. We need to open a pdf before we can ask information about it.",
)

In [72]:
get_page_details_tool = Tool.from_function(
    name='get_page_details_tool',
    func=lambda page_number: doc[int(page_number) - 1].get_text(),
    description="Gets the text from a page in the pdf. Takes in the page number and returns the text in the page. We need to open the pdf using open_pdf_tool before we can ask information about it.",
)

In [73]:
tools = [open_pdf_tool, get_page_details_tool]

In [74]:
pdf_analyzer_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        You are a research agent. You will be given a pdf file. This file can only be read by the tools provided to you. The pdf file has multiple pages 
        with information in it. You need to read the file and create a brief summary of the information in the file.

        Go through the pdf page by page and generate a summary of each page.

        Return a result in the markdown format.
        """
    ),
    ("human", "Pdf file: {pdf_file}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [75]:
pdf_analyzer_prompt.messages[2].__dict__

{'variable_name': 'agent_scratchpad', 'optional': False, 'n_messages': None}

In [79]:
def agent_scratchpad_formatter(x):
    return format_to_openai_tool_messages(x["intermediate_steps"])

llm = llm.bind_tools(tools)

def human_approval(msg: AIMessage, *args, **kwargs):
    print(args, kwargs)
    next_tool_action = parse_ai_message_to_tool_action(msg)
    if isinstance(next_tool_action, AgentFinish):
        input_msg = (
            f"Are you happy with the output? If no, tell us what you would like to change. If yes, type 'yes' to approve the agent result"
        )
        resp = input(input_msg)
        if resp.lower() not in ("yes", "y"):
            raise ValueError(f"Agent success!")
    return msg

agent = (
    {
        "pdf_file": lambda x: x["pdf_file"],
        "agent_scratchpad": agent_scratchpad_formatter,
    }
    | pdf_analyzer_prompt
    | llm
    | human_approval
    | OpenAIToolsAgentOuttParser()
)

In [80]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False,)

In [81]:
pdf_file = "tata_motors_report.pdf"
research_agent_result = await agent_executor.ainvoke({"pdf_file": pdf_file}, config={"callbacks": [langfuse_handler]})


with open(f"""output/{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}_research_report.md""", 'w') as file:
    file.write(research_agent_result['output'])



> Entering new AgentExecutor chain...
() {}

Invoking: `open_pdf_tool` with `tata_motors_report.pdf`


Opened tata_motors_report.pdf pdf file for analysis. The document has 6 pages.() {}

Invoking: `get_page_details_tool` with `1`



Invoking: `get_page_details_tool` with `2`



Invoking: `get_page_details_tool` with `3`



Invoking: `get_page_details_tool` with `4`



Invoking: `get_page_details_tool` with `5`



Invoking: `get_page_details_tool` with `6`


 
 
 
ICICI Securities | Retail Research 
 
2 
ICICI Direct Research 
 
Result Update | Tata Motors 
Q4FY24 Earnings Conference Call Highlights 
Jaguar Land Rover 
• 
Order Book: The order book at the end of FY24 was around 133,000 
vehicles, out of which 76% of orders were from Range Rover, Range Rover 
Sport, and Defender. Demand for Range Rover Sport Vehicle more than 
doubled to 4,099 units in FY24, with an average price point of ~ £202,000. 
This also includes the sale of 20 Range Rover SV Bespoke sadaf editions 
selling out